In [1]:
!pip install eli5
!pip install spacy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.2/216.2 kB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for eli5: filename=eli5-0.13.0-py2.py3-none-any.whl size=107719 sha256=90bd6718e18638494ce05613eef4930e35beb827f76640734a1be7aa5c1aa4c9
  Stored in directory: /root/.cache/pip/wheels/b8/58/ef/2cf4c306898c2338d51540e0922c8e0d6028e07007085c0004
Successfully built eli5


In [2]:
import pandas as pd
import numpy as np
import re
import string
import spacy
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction._stop_words import ENGLISH_STOP_WORDS

from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report
from sklearn.naive_bayes import MultinomialNB

from sklearn.feature_extraction.text import TfidfTransformer,  TfidfVectorizer
from sklearn.pipeline import Pipeline

from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV

import eli5

In [3]:
from google.colab import drive
drive.mount('/content/drive')
pd.set_option('max_colwidth', 400)
# load data
data = pd.read_csv('/content/drive/MyDrive/NLP/spam_or_not_spam.csv', encoding='iso-8859-1')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
print(data['email'].head())
print(data['label'].value_counts())

0     date wed NUMBER aug NUMBER NUMBER NUMBER NUMBER NUMBER from chris garrigues cwg dated NUMBER NUMBERfaNUMBERd deepeddy com message id NUMBER NUMBER tmda deepeddy vircio com i can t reproduce this error for me it is very repeatable like every time without fail this is the debug log of the pick happening NUMBER NUMBER NUMBER pick_it exec pick inbox list lbrace lbrace subject ftp rbrace rbrace NU...
1    martin a posted tassos papadopoulos the greek sculptor behind the plan judged that the limestone of mount kerdylio NUMBER miles east of salonika and not far from the mount athos monastic community was ideal for the patriotic sculpture as well as alexander s granite features NUMBER ft high and NUMBER ft wide a museum a restored amphitheatre and car park for admiring crowds are planned so is thi...
2    man threatens explosion in moscow thursday august NUMBER NUMBER NUMBER NUMBER pm moscow ap security officers on thursday seized an unidentified man who said he was armed with explosives

In [5]:
nlp = spacy.load("en_core_web_sm")
stopwords = nlp.Defaults.stop_words
print(f'Spacy english stopwords size: {len(stopwords)}', end='\n\n')
' '.join(stopwords)

Spacy english stopwords size: 326



"several further thence indeed the doing could 'd does twenty most amount move very neither or either did my thereby hence on nothing noone many our for a towards became formerly its when herself although once because seem seems which before put latter now with ours how than will at there show something around everywhere without otherwise and empty next keep do bottom get side since re are ‘ll fifty until whoever cannot own every less who though take anywhere am as behind not never whole in done another well from but whereupon ‘re myself whereafter his third whatever least himself above first she again along wherein her none yet whenever ever after almost ‘d often eleven wherever last using such an why say whom per seemed also sometimes five was elsewhere two hers ’ll full someone whose us namely is amongst ’d anyway some all eight regarding were call about beforehand nobody under various only hereby me throughout would back always herein n't over n‘t yours nine this anyhow 'm anyone w

In [6]:
data['label'].head()
print(data.iloc[2997])
list(data)

email     thank you for shopping with us gifts for all occasions free gift with NUMBER NUMBER purchase for a limited time only receive this NUMBER plush santa bear free with your purchase of NUMBER NUMBER or more when your order totals NUMBER NUMBER or more order must be NUMBER NUMBER or more before shipping and handling this santa bear is added to your cart for free while supplies last mary s store wo...
label                                                                                                                                                                                                                                                                                                                                                                                                                  1
Name: 2997, dtype: object


['email', 'label']

In [7]:
data['email'] = data['email'].astype(str)
data['cleaned_text'] = data['email'].apply(
    lambda x: ' '.join(
        token.lemma_.lower() for token in nlp(x) if
        not token.is_stop
        and not token.is_punct
        and not token.is_digit
        and not token.like_email
        and not token.like_num
        and not token.is_space
    )
)
data.sample(5)

,email,label,cleaned_text
2841,are you interested in making some extra money on the internet well have i got something for you last week i made NUMBER i am offering you NUMBER yes NUMBER web sites that have already been made and are waiting for you to put up to make money with there is also a few videos that are included that tell you exactly what you have to do to be successful i am going to also offer you the rights to th...,1,interested make extra money internet get week number offer number yes number web site wait money video include tell exactly successful go offer right web page number ebook sell ebook aren t ebook book money internet sell package deal short time low price number number friend crazy web site worth number number number ebook receive worth number web page url come look question feel free email d l...
455,lol they re not doomed at all thousands of men wear cbNUMBERs to work every day along with other chastity devices they are not just decorations you will not get a woody or have an orgasm until your keyholder allows you to anyhow i know better than to have this type of conversation with a nilla but these men wear these happily and consensually there is little power struggle in the lifestyles of...,0,lol doom thousand man wear cbnumber work day chastity device decoration woody orgasm keyholder allow know well type conversation nilla man wear happily consensually little power struggle lifestyle people use power exchange instead lot variation definition s gloria brame s site power exchange consensual transfer power submissive dominant exchange take place return energy dominant empower submis...
62,bob we are a commercial company who host around NUMBER web sites on each of it servers we can t swap with you because we need someone in a similar position thank you javier el viernes NUMBER agosto NUMBER a las NUMBER NUMBER am bob musser escribiÃ³ i m using simple dns from jhsoft we support only a few web sites and i d like to swap secondary services with someone in a similar position we have...,0,bob commercial company host number web site server t swap need similar position thank javier el viernes number agosto number las number number bob musser escribiã³ m simple dns jhsoft support web site d like swap secondary service similar position static ip dsl line number number set web sql mail dns server say host number web site web dns traffic lightly load apc battery backup seldom d like ...
2638,hyperlink hyperlink hyperlink hyperlink,1,hyperlink hyperlink hyperlink hyperlink
2397,url URL date NUMBER NUMBER NUMBERtNUMBER NUMBER NUMBER NUMBER NUMBER cartoon steve bell on the conservative party conference,0,url url date number number numbertnumber number number number number cartoon steve bell conservative party conference


In [8]:
print(data['email'].iloc[2998])

 the famous ebay marketing e course learn to sell with the complete ebay auction marketing e course here s your chance to join the online selling revolution and earn a full time income our ebay marketing e course will show you how to create huge profits selling on ebay do you sell on ebay if so you could be making up to NUMBER NUMBER per month this is no hype and no scam receiving over NUMBER NUMBER billion page views per month ebay is the ultimate venue for selling virtually anything and making huge profits with almost no effort but you have to know what to sell and how to sell that s where i come in as a leading expert in internet marketing and the owner of several profitable auction based businesses the manual that i have written provides easy to understand and detailed instructions for maximizing your profits with selling strategies that are proven winners if you ve read any other books on ebay you know that all of them are designed for the computer idiot and the auction novice the

In [9]:
x_train, x_test, y_train, y_test = train_test_split(data['cleaned_text'], data['label'], random_state=30)

In [10]:
# CountVectorize -> DecisionTree

vectorizer = CountVectorizer(max_df=0.7, min_df=0.003) # max_df фильтрует corpus-specific stop words
x_train_vectorized = vectorizer.fit_transform(x_train)
x_test_vectorized = vectorizer.transform(x_test)

clf = DecisionTreeClassifier().fit(x_train_vectorized, y_train)
preds = clf.predict(x_test_vectorized)
print(classification_report(y_test, preds))

              precision    recall  f1-score   support

           0       0.98      0.96      0.97       637
           1       0.80      0.89      0.85       113

    accuracy                           0.95       750
   macro avg       0.89      0.93      0.91       750
weighted avg       0.95      0.95      0.95       750



In [11]:
# TF-IFD vectorize -> DecisionTree
pipe = Pipeline(
    steps=[
        ('tfidf', TfidfVectorizer()),
        ('clf', DecisionTreeClassifier())
    ]
).fit(x_train, y_train)
preds = pipe.predict(x_test)
print(classification_report(y_test, preds))

              precision    recall  f1-score   support

           0       0.98      0.97      0.97       637
           1       0.83      0.88      0.86       113

    accuracy                           0.96       750
   macro avg       0.91      0.93      0.92       750
weighted avg       0.96      0.96      0.96       750



In [12]:
# Count vectorize -> LogReg
vectorizer = CountVectorizer(max_df=0.7, min_df=0.003) # max_df фильтрует corpus-specific stop words
x_train_vectorized = vectorizer.fit_transform(x_train)
x_test_vectorized = vectorizer.transform(x_test)

logreg = LogisticRegression().fit(x_train_vectorized, y_train)
preds = logreg.predict(x_test_vectorized)
print(classification_report(y_test, preds))

              precision    recall  f1-score   support

           0       0.99      1.00      0.99       637
           1       1.00      0.94      0.97       113

    accuracy                           0.99       750
   macro avg       0.99      0.97      0.98       750
weighted avg       0.99      0.99      0.99       750



In [13]:
# TF-IFD vectorize -> LogReg
pipe = Pipeline(
    steps=[
        ('tfidf', TfidfVectorizer()),
        ('clf', LogisticRegression())
    ]
).fit(x_train, y_train)
preds = pipe.predict(x_test)
print(classification_report(y_test, preds))

              precision    recall  f1-score   support

           0       0.96      1.00      0.98       637
           1       1.00      0.78      0.88       113

    accuracy                           0.97       750
   macro avg       0.98      0.89      0.93       750
weighted avg       0.97      0.97      0.96       750



In [14]:
# Count vectorize -> MultinominalNB

vectorizer = CountVectorizer(max_df=0.7, min_df=0.003) # max_df фильтрует corpus-specific stop words
x_train_vectorized = vectorizer.fit_transform(x_train)
x_test_vectorized = vectorizer.transform(x_test)

clf = MultinomialNB().fit(x_train_vectorized, y_train)
preds = clf.predict(x_test_vectorized)
print(classification_report(y_test, preds))

              precision    recall  f1-score   support

           0       0.99      1.00      1.00       637
           1       0.99      0.96      0.97       113

    accuracy                           0.99       750
   macro avg       0.99      0.98      0.98       750
weighted avg       0.99      0.99      0.99       750



In [15]:
# TF-IFD vectorize -> MultinominalNB
pipe = Pipeline(
    steps=[
        ('tfidf', TfidfVectorizer()),
        ('clf', MultinomialNB())
    ]
).fit(x_train, y_train)
preds = pipe.predict(x_test)
print(classification_report(y_test, preds))

              precision    recall  f1-score   support

           0       0.90      1.00      0.95       637
           1       1.00      0.37      0.54       113

    accuracy                           0.91       750
   macro avg       0.95      0.69      0.74       750
weighted avg       0.91      0.91      0.89       750



In [16]:
# LogReg

pipe = Pipeline(
    steps=[
        ('counter', CountVectorizer()),
        ('tfidf', TfidfTransformer()),
        ('clf', LogisticRegression())
    ]
)
parameter_grid = {
    "counter__max_df": np.linspace(0.3, 0.7, 10),
    "counter__min_df": [0.0, 0.001, 0.003, 0.005],
    "counter__ngram_range": ((1, 1), (1, 2)),  # слова или биграммы
    "tfidf__norm": ("l1", "l2"),
    "clf__C": np.linspace(0.1, 1, 10),
}

grid_search = HalvingGridSearchCV(
    pipe,
    param_grid=parameter_grid,
    n_jobs=-1,
    verbose=1,
    cv=2,
    scoring='f1',
    random_state=42,
)
grid_search.fit(x_train, y_train)
grid_search.best_estimator_['clf']
eli5.show_weights(
    estimator=grid_search.best_estimator_['clf'],
    feature_names= list(grid_search.best_estimator_['counter'].get_feature_names_out()),
    top=(50, 5)
)

n_iterations: 6
n_required_iterations: 7
n_possible_iterations: 6
min_resources_: 8
max_resources_: 2250
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 1600
n_resources: 8
Fitting 2 folds for each of 1600 candidates, totalling 3200 fits


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
1600 fits failed out of a total of 3200.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
1600 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/pipeline.py", line 405, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py", line 1241, in fit
    raise ValueError(
ValueError: This solver needs samples 

----------
iter: 1
n_candidates: 534
n_resources: 24
Fitting 2 folds for each of 534 candidates, totalling 1068 fits


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
534 fits failed out of a total of 1068.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
534 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/pipeline.py", line 405, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py", line 1241, in fit
    raise ValueError(
ValueError: This solver needs samples of

----------
iter: 2
n_candidates: 178
n_resources: 72
Fitting 2 folds for each of 178 candidates, totalling 356 fits


...

----------
iter: 4
n_candidates: 20
n_resources: 648
Fitting 2 folds for each of 20 candidates, totalling 40 fits


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:952: UserWarning: One or more of the test scores are non-finite: [       nan        nan        nan ... 0.08475073 0.         0.3030303 ]
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:952: UserWarning: One or more of the train scores are non-finite: [       nan        nan        nan ... 0.0998557  0.         0.47375279]
  warnings.warn(


----------
iter: 5
n_candidates: 7
n_resources: 1944
Fitting 2 folds for each of 7 candidates, totalling 14 fits


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:952: UserWarning: One or more of the test scores are non-finite: [       nan        nan        nan ... 0.4597561  0.51794177 0.77398194]
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:952: UserWarning: One or more of the train scores are non-finite: [       nan        nan        nan ... 0.54658606 0.60851064 0.86605511]
  warnings.warn(


Weight?,Feature
+4.955,hyperlink
+3.413,click
+2.591,free
+2.256,money
+1.949,remove
+1.827,email
+1.581,mail
+1.559,receive
+1.553,visit
+1.490,wish


In [ ]:
# LogReg

plt.rcParams["figure.figsize"] = (12, 8)
results = pd.DataFrame(grid_search.cv_results_)
results["params_str"] = results.params.apply(str)
results.drop_duplicates(subset=("params_str", "iter"), inplace=True)
mean_scores = results.pivot(
    index="iter",
    columns="params_str",
     values="mean_test_score",
)
ax = mean_scores.plot(legend=False, alpha=0.6)

labels = [
    f"iter={i}\nn_samples={grid_search.n_resources_[i]}\nn_candidates={grid_search.n_candidates_[i]}"
    for i in range(grid_search.n_iterations_)
]

ax.set_xticks(range(grid_search.n_iterations_))
ax.set_xticklabels(labels, rotation=45, multialignment="left")
ax.set_title("Scores of candidates over iterations")
ax.set_ylabel("Mean test score", fontsize=15)
ax.set_xlabel("Iterations", fontsize=15)
plt.tight_layout()
plt.grid()
plt.show()

In [ ]:
# MultinomialNB

pipe = Pipeline(
    steps=[
        ('counter', CountVectorizer()),
        ('tfidf', TfidfTransformer()),
        ('clf', MultinomialNB())
    ]
)
parameter_grid = {
    "counter__max_df": np.linspace(0.3, 0.7, 10),
    "counter__min_df": [0.0, 0.001, 0.003, 0.005],
    "counter__ngram_range": ((1, 1), (1, 2)),  # слова или биграммы
    "tfidf__norm": ("l1", "l2"),
    "clf__alpha": np.linspace(0.1, 1, 10),
}

grid_search = HalvingGridSearchCV(
    pipe,
    param_grid=parameter_grid,
    n_jobs=-1,
    verbose=1,
    cv=2,
    scoring='f1',
    random_state=42,
)
grid_search.fit(x_train, y_train)
grid_search.best_estimator_['clf']
eli5.show_weights(
    estimator=grid_search.best_estimator_['clf'],
    feature_names= list(grid_search.best_estimator_['counter'].get_feature_names_out()),
    top=(50, 5)
)

plt.rcParams["figure.figsize"] = (12, 8)
results = pd.DataFrame(grid_search.cv_results_)
results["params_str"] = results.params.apply(str)
results.drop_duplicates(subset=("params_str", "iter"), inplace=True)
mean_scores = results.pivot(
    index="iter",
    columns="params_str",
     values="mean_test_score",
)
ax = mean_scores.plot(legend=False, alpha=0.6)

labels = [
    f"iter={i}\nn_samples={grid_search.n_resources_[i]}\nn_candidates={grid_search.n_candidates_[i]}"
    for i in range(grid_search.n_iterations_)
]

ax.set_xticks(range(grid_search.n_iterations_))
ax.set_xticklabels(labels, rotation=45, multialignment="left")
ax.set_title("Scores of candidates over iterations")
ax.set_ylabel("Mean test score", fontsize=15)
ax.set_xlabel("Iterations", fontsize=15)
plt.tight_layout()
plt.grid()
plt.show()

n_iterations: 6
n_required_iterations: 7
n_possible_iterations: 6
min_resources_: 8
max_resources_: 2250
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 1600
n_resources: 8
Fitting 2 folds for each of 1600 candidates, totalling 3200 fits
